In [64]:
%matplotlib inline

import h5py
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import scipy
from collections import Counter
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display
from numpy.random import choice
from numpy import linalg as LA
from scipy import optimize
from scipy.special import expit

mpl.style.use('seaborn')

In [2]:
def get_accuracy(y_true, y_pred): 
    """
    Calculate the accuracy score.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sum(y_true == y_pred)/len(y_true)

def flat(data):
    """
    Flatten images from 2D to 1D in the data array. 
    """
    return data.reshape(data.shape[0], data.shape[1]**2)

In [3]:
# load training data and labels
with h5py.File('images_training.h5','r') as H:
    data = np.copy(H['data'])
    data = np.array([x/np.amax(x) for x in data]) # normalize images
with h5py.File('labels_training.h5','r') as H:
    label = np.copy(H['label'])
    label = np.array(label, dtype=np.int32)

In [44]:
# load testing data and labels
with h5py.File('images_testing.h5','r') as H:
    data_test = np.copy(H['data'])
    data_test = np.array([x/np.amax(x) for x in data_test]) # normalize images
with h5py.File('labels_testing_2000.h5','r') as H:
    label_test = np.copy(H['label'])
    label_test = np.array(label_test, dtype=np.int32)

## Singular Value Decomposition 

In [5]:
k_svd = 10 # number of singular values to save

In [6]:
# Singular Value Decomposition on training images

u, s, vh = LA.svd(data)
data_svd = np.array([u[i][:,:k_svd] @ np.diag(s[i][:k_svd]) @ vh[i][:k_svd:,] for i in range(s.shape[0])])

In [7]:
# Singular Value Decomposition on testing images

u, s, vh = LA.svd(data_test)
data_test_svd = np.array([u[i][:,:k_svd] @ np.diag(s[i][:k_svd]) @ vh[i][:k_svd:,] for i in range(s.shape[0])])

## Principal Component Analysis

Note: PCA is applied to the entire dataset, while SVD is applied to each image. 

In [65]:
def PCA(n_components, data):
    """
    Principal Component Analysis
    
    Parameters
    ----------
    
    n_components: int
        Number of principal components to use
        
    data : array, shape = (n_samples, n_features)
        Data on which to perform PCA
        
    Returns 
    -------
    
    X_PCA: array, shape = (n_samples, n_components)
        PCA transformed data
    
    """
    n_components = n_components
    data = data
    X = data - np.mean(data, axis=0) # data matrix normalized by mean
    S = np.cov(X.T) # compute covariance matrix of X
    L, V = LA.eig(S) # L - array of eigenvalues; V - matrix of eigenvectors
    V = V[:,np.argsort(-L)][:,:n_components] # sort eigenvectors by descending order on eigenvalues
    L = -np.sort(-L)[:n_components] # sort eigenvalues by descending order
    X_PCA = V.T @ X.T
    X_PCA = X_PCA.T
    return X_PCA

In [66]:
n_components = 20
X_PCA = PCA(n_components=n_components, data=np.append(flat(data), flat(data_test), axis=0))
data_pca = X_PCA[:data.shape[0]]
data_test_pca = X_PCA[-data_test.shape[0]:]
print(data_pca.shape)
print(data_test_pca.shape)

(30000, 20)
(5000, 20)


## Gaussian Process Classification

In [53]:
def gen_sum2(x0, x1):
    """
    Generator for computing individual sum of squares entries. 
    """
    for i in range(x0.shape[0]):
        for j in range(x1.shape[0]):
            yield i, j, np.sum(np.square(x0[i] - x1[j]))

def kernel(x0, x1, theta, deriv=False):
    """
    Calculate the kernel matrix. 
    
    Parameters
    ----------
    
    x0, x1: arrays, shape = (n_samples, n_features)
        input arrays
    
    theta: array, shape = (3,)
        hyperparameters for kernel function
    
    deriv: boolean
        whether to return derivatives of kernel matrix wrt theta
    
    Returns
    -------
    
    k: array, shape = (x0.shape[0], x1.shape[0])
        kernel matrix, optional
        returned if deriv == False
    
    K: array, shape = (len(theta)+1, x0.shape[0], x1.shape[0])
        kernel matrix and derivatives wrt theta, optional
        returned if deriv == True
    """
    # Take the exponent of theta to ensure positive values
    theta = np.exp(theta)
    
    # Sum of squares for input values
    sum2 = np.zeros((x0.shape[0], x1.shape[0]))
    for i, j, s in gen_sum2(x0, x1):
        sum2[i][j] = s*theta[1]

    k = theta[0] * np.exp(-0.5*sum2)
    
    if deriv:
        K = np.zeros((len(theta)+1, x0.shape[0], x1.shape[0]))
        # K[:,:,0] is the original covariance matrix
        K[0] = k + theta[2]*np.eye(x0.shape[0], x1.shape[0])
        K[1] = k
        K[2] = -0.5*sum2*k
        K[3] = theta[2]*np.eye(x0.shape[0], x1.shape[0])
        return K
    else:
        return k + theta[2]*np.eye(x0.shape[0], x1.shape[0])

In [51]:
def logPosterior(theta, *args):
    """
    Calculate the negative posterior log likelihood. 
    
    Parameters
    ----------
    
    theta: array, shape = (3,)
        hyperparameters for kernel function
    
    data: array, shape = (n_samples, n_features)
        input array
    
    t: array, shape = (1, n_samples)
        label column
    
    Returns:
    --------
    
    -logp: float
        negative of posterior log likelihood
    
    
    """
    data, t = args
    k = kernel(data, data, theta, deriv=False)
    L = LA.cholesky(k)
    alpha = LA.solve(L.T, LA.solve(L, t))
    logp = -0.5*t.T @ alpha - np.sum(np.log(np.diag(L))) - 0.5*data.shape[0]*np.log(2.0*np.pi)
    return -logp

def gradLogPosterior(theta, *args):
    """
    Calculate the gradient of the posterior log likelihood. 
    
    theta: array, shape = (3,)
        hyperparameters for kernel function
    
    data: array, shape = (n_samples, n_features)
        input array
    
    t: array, shape = (1, n_samples)
        label column
    
    Returns:
    --------
    
    -dlogpdtheta: array, shape = (3,)
        negative of posterior log likelihood gradient
    
    """
    data, t = args
    K = kernel(data, data, theta, deriv=True)

    L = LA.cholesky(K[0])
    k_inv = LA.solve(L.T, LA.solve(L, np.eye(data.shape[0])))
    dlogpdtheta = np.zeros(len(theta))
    for d in range(1, len(theta)+1):
        dlogpdtheta[d-1] = 0.5*t.T@k_inv@K[d]@k_inv@t - 0.5*np.trace(k_inv@K[d])

    return -dlogpdtheta

In [76]:
%%time

sub = choice(data.shape[0], 10000)
train = flat(data)[sub]
train_label = label[sub]
test = flat(data_test)[:2000]
y_true = label_test

theta = np.log([1, 0.005, 1e-10])
values = np.array(list(set(train_label)))
P = []
for i in values:
    # create binary class labels for class i
    t = np.zeros(train_label.shape[0], dtype=int)
    t[train_label == i] = 1
    t = t.T
    print('Training Class {}'.format(i))
    print(Counter(t))
    
    K = kernel(train, train, theta, deriv=False)
    K_s = kernel(test, train, theta, deriv=False)
    L = LA.cholesky(K)
    alpha = LA.solve(L.T, LA.solve(L, t))
    mu = K_s @ alpha
    logit = expit(mu)
    prob = (logit-np.amin(logit))/(np.ptp(expit(mu)))
    P.append(prob)
P = np.array(P)
y_pred = np.argmax(P, axis=0)
y_pred = np.array([values[i] for i in y_pred])
print(get_accuracy(y_pred, y_true))

Training Class 0
Counter({0: 9049, 1: 951})
Training Class 1
Counter({0: 8986, 1: 1014})
Training Class 2
Counter({0: 8994, 1: 1006})
Training Class 3
Counter({0: 9019, 1: 981})
Training Class 4
Counter({0: 8928, 1: 1072})
Training Class 5
Counter({0: 8998, 1: 1002})
Training Class 6
Counter({0: 8998, 1: 1002})
Training Class 7
Counter({0: 9047, 1: 953})
Training Class 8
Counter({0: 9023, 1: 977})
Training Class 9
Counter({0: 8958, 1: 1042})
0.85
Wall time: 3h 44min 20s


In [77]:
with h5py.File('predicted_labels.h5','w') as H:
    H.create_dataset('label',data=y_pred)